In [1]:
import pandas as pd

In [2]:
icdCodesProc = pd.read_csv("d_icd_procedures.csv")

In [3]:
procedures = pd.read_csv("procedures_icd.csv")

In [4]:
procedures = procedures[procedures.icd_version != 9]

In [5]:
#specified_codes = ['0VT00ZZ', '0DBJ0ZZ', '0UT90ZZ', '0SRC0J9', '0FB40ZZ', '0JH604Z']
#filtered_rows = procedures[procedures['icd_code'].isin(specified_codes)]
filtered_rows = procedures[procedures['icd_code'].str.startswith(('0DTJ', '0FT4'))] #appe or chole
filtered_rows

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
340,10004606,29242151,1,2159-02-25,0FT44ZZ,10
519,10006431,28484351,1,2128-06-12,0FT44ZZ,10
790,10012206,23961896,1,2127-07-10,0FT44ZZ,10
1498,10022373,29953960,1,2150-06-30,0FT40ZZ,10
1621,10024451,21654679,3,2118-08-12,0FT40ZZ,10
...,...,...,...,...,...,...
666507,19960203,25978628,4,2140-10-01,0FT40ZZ,10
666766,19965286,26100445,1,2171-12-16,0FT44ZZ,10
667489,19975995,26284923,1,2111-03-10,0DTJ4ZZ,10
668198,19987702,24320929,1,2131-05-15,0FT40ZZ,10


In [6]:
discharge = pd.read_csv('discharge.csv')

In [7]:
discharge.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...


In [8]:
from readability import Readability

# Assuming df_procedures and df_discharge are your two DataFrames
procWNotes = pd.merge(filtered_rows, discharge[['hadm_id', 'text']], on='hadm_id')

def calculate_fk_score(text):
    score = 0
    try:
        text = text.split("Discharge Instructions",1)[1]
        r = Readability(text)
        score = r.flesch_kincaid().score
    except Exception as e:
        print(e)
        score = -1
    return score


procWNotes['flesch_kincaid_score'] = procWNotes['text'].apply(calculate_fk_score)
procWNotes = procWNotes.drop('text', axis=1)


100 words required.
100 words required.
100 words required.
list index out of range
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
list index out of range
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
list index out of range
100 words required.
list index out of range
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
list index out of range
100 words required.
100 words required.
100 words required.
100 words required.
list index out of range
100 words required.
100 words required.
list index out of range
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words required.
100 words re

In [9]:
procWNotes = procWNotes[procWNotes['flesch_kincaid_score'] != -1]
procWNotes

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version,flesch_kincaid_score
0,10004606,29242151,1,2159-02-25,0FT44ZZ,10,8.121622
1,10012206,23961896,1,2127-07-10,0FT44ZZ,10,6.949348
2,10022373,29953960,1,2150-06-30,0FT40ZZ,10,9.417138
3,10024451,21654679,3,2118-08-12,0FT40ZZ,10,7.984823
4,10025862,21206487,3,2173-08-09,0FT40ZZ,10,7.922595
...,...,...,...,...,...,...,...
1511,19960203,25978628,4,2140-10-01,0FT40ZZ,10,8.937806
1512,19965286,26100445,1,2171-12-16,0FT44ZZ,10,6.599717
1513,19975995,26284923,1,2111-03-10,0DTJ4ZZ,10,6.797813
1514,19987702,24320929,1,2131-05-15,0FT40ZZ,10,8.878576


In [10]:
admissions = pd.read_csv("admissions.csv")

In [11]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [15]:
import pandas as pd

# Assuming df1 is your procedures table and df2 is your admittances table

# Convert chartdate and admittime to datetime
procWNotes['chartdate'] = pd.to_datetime(procWNotes['chartdate'])
admissions['admittime'] = pd.to_datetime(admissions['admittime'])

# Merge the two dataframes on subject_id
merged_df = pd.merge(procWNotes, admissions, on='subject_id', suffixes=('_proc', '_adm'), how='left')

# Filter rows where admittime is within 30 days after chartdate but not same hadm_id
filtered_df = merged_df[(merged_df['admittime'] > merged_df['chartdate']) & 
                        (merged_df['admittime'] <= merged_df['chartdate'] + pd.Timedelta(days=30)) &
                        (merged_df['hadm_id_proc'] != merged_df['hadm_id_adm'])]

# Sort by the difference in days and drop duplicates to keep the closest date
filtered_df['date_diff'] = (filtered_df['admittime'] - filtered_df['chartdate']).dt.days
filtered_df = filtered_df.sort_values(by='date_diff').drop_duplicates(['subject_id', 'chartdate', 'icd_code'], keep='first')

# Now filtered_df contains the desired rows
print(filtered_df)


      subject_id  hadm_id_proc  seq_num  chartdate icd_code  icd_version  \
3975    18875996      25162160        1 2148-04-21  0DTJ4ZZ           10   
3033    16831849      26479392        1 2156-07-25  0FT44ZZ           10   
2341    15355404      22675141        1 2168-11-27  0DTJ4ZZ           10   
462     11113542      26090767        1 2187-08-18  0FT44ZZ           10   
2945    16557293      24963530        1 2120-12-27  0FT44ZZ           10   
...          ...           ...      ...        ...      ...          ...   
3928    18817034      28303195        7 2177-10-17  0FT40ZZ           10   
2787    16230838      20824785        1 2181-03-02  0FT44ZZ           10   
3473    17887687      24438766        1 2184-09-07  0DTJ4ZZ           10   
2650    15926903      25334303        1 2179-09-16  0DTJ4ZZ           10   
330     10889053      23363256       10 2137-04-09  0DTJ0ZZ           10   

      flesch_kincaid_score  hadm_id_adm           admittime  \
3975              8.1748

/var/folders/4x/vzxb2kss04955dfkk_r657bm0000gn/T/ipykernel_8369/2903657242.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['date_diff'] = (filtered_df['admittime'] - filtered_df['chartdate']).dt.days


In [25]:
# List of subject_ids to remove
ids_to_remove = filtered_df['subject_id'].tolist()

# Filter out rows where subject_id is in ids_to_remove
comparedDF = procWNotes[~procWNotes['subject_id'].isin(ids_to_remove)]

print(filtered_df)

      subject_id   hadm_id  seq_num  chartdate icd_code  icd_version  \
1       10012206  23961896        1 2127-07-10  0FT44ZZ           10   
3       10024451  21654679        3 2118-08-12  0FT40ZZ           10   
5       10039870  22600344        1 2176-04-22  0DTJ0ZZ           10   
6       10052530  27361644        1 2186-01-24  0DTJ4ZZ           10   
7       10052926  29714792        1 2123-07-21  0FT44ZZ           10   
...          ...       ...      ...        ...      ...          ...   
1509    19931286  24781526        4 2198-02-12  0FT40ZZ           10   
1510    19955582  26593491        1 2139-10-14  0DTJ4ZZ           10   
1511    19960203  25978628        4 2140-10-01  0FT40ZZ           10   
1512    19965286  26100445        1 2171-12-16  0FT44ZZ           10   
1515    19993764  23707485        2 2167-08-29  0FT44ZZ           10   

      flesch_kincaid_score  
1                 6.949348  
3                 7.984823  
5                 8.450618  
6                 6

In [26]:
print(comparedDF)


      subject_id   hadm_id  seq_num  chartdate icd_code  icd_version  \
1       10012206  23961896        1 2127-07-10  0FT44ZZ           10   
3       10024451  21654679        3 2118-08-12  0FT40ZZ           10   
5       10039870  22600344        1 2176-04-22  0DTJ0ZZ           10   
6       10052530  27361644        1 2186-01-24  0DTJ4ZZ           10   
7       10052926  29714792        1 2123-07-21  0FT44ZZ           10   
...          ...       ...      ...        ...      ...          ...   
1509    19931286  24781526        4 2198-02-12  0FT40ZZ           10   
1510    19955582  26593491        1 2139-10-14  0DTJ4ZZ           10   
1511    19960203  25978628        4 2140-10-01  0FT40ZZ           10   
1512    19965286  26100445        1 2171-12-16  0FT44ZZ           10   
1515    19993764  23707485        2 2167-08-29  0FT44ZZ           10   

      flesch_kincaid_score  
1                 6.949348  
3                 7.984823  
5                 8.450618  
6                 6

In [28]:
readmits = filtered_df['flesch_kincaid_score'].tolist()
compared = comparedDF['flesch_kincaid_score'].tolist()

In [30]:
from scipy.stats import ttest_ind

print(ttest_ind(readmits, compared))

Ttest_indResult(statistic=2.1936958796030916, pvalue=0.028416861291099667)
